In [1]:
import numpy as np
from itertools import combinations

In [2]:
def are_collinear(p1, p2, p3):
  """Returns True if the three points are collinear."""
  """Adapt from https://github.com/kitft/funsearch"""
  x1, y1 = p1
  x2, y2 = p2
  x3, y3 = p3
  return (y1 - y2) * (x1 - x3) == (y1 - y3) * (x1 - x2)

class N3il: # Class of No-3-In-Line
    def __init__(self, grid_size):
        self.row_count = grid_size[0]
        self.column_count = grid_size[1]
        self.action_size = self.row_count * self.column_count
    
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    def get_next_state(self, state, action):
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = 1

        return state
    
    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
    
    def check_collinear(self, state, action):
        row = action // self.column_count
        column = action % self.column_count
        state_next = state.copy()
        state_next[row, column] = 1

        # Get the coordinates of all points with value 1
        coords = np.argwhere(state_next == 1) 
        # Convert to list of tuples (optional)
        # coord_list = [tuple(coord) for coord in coords]
        # Get all combinations of 3 points
        triples = list(combinations(coords, 3))

        number_of_collinear_triples = 0 
        # CONSIDER give configs with more collinear triples more "punishment"
        # But it may not be a good idea, consider in 3x3 case, when we got
        # 6 points, adding a points would results in 3 triples collinear, but
        # it doesn't mean the config of 6 points is not good.
        for triple in triples:
            if are_collinear(triple[0], triple[1], triple[2]):
                number_of_collinear_triples += 1
        
        return number_of_collinear_triples
    
    def get_value_and_terminated(self, state):
        value = np.sum(state.reshape(-1) == 1)
        return value, True # Return TRUE if the configuration involves any 3 points collinear

In [3]:
n3il = N3il(grid_size=(3,3))

state = n3il.get_initial_state()

while True:
    print(state)

    valid_moves = n3il.get_valid_moves(state)
    list_valid_moves = [i for i in range(n3il.action_size) if valid_moves[i] == 1]
    print("valid moves: ", list_valid_moves)
    
    action = int(input("Please give a point: "))
    if action not in list_valid_moves:
        print("This spot is occupied. Action is not valid.")
        continue

    n_of_collinear_triples = n3il.check_collinear(state, action)

    if n_of_collinear_triples > 0:
        value, _ = n3il.get_value_and_terminated(state)
        print("------------------------------------------------------------------")
        print(f"Trial Terminated with {value} points. Final valid configuration:")
        print(state)
        print(f"The point you give causes {n_of_collinear_triples} triples of 3 points collinear:")
        print(n3il.get_next_state(state, action))

        break

    state = n3il.get_next_state(state, action)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves:  [0, 1, 2, 3, 4, 5, 6, 7, 8]
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves:  [1, 2, 3, 4, 5, 6, 7, 8]
[[1. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
valid moves:  [2, 3, 4, 5, 6, 7, 8]
------------------------------------------------------------------
Trial Terminated with 2 points. Final valid configuration:
[[1. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
The point you give causes 1 triples of 3 points collinear:
[[1. 1. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
